# **Bài tập Colab**
# **Thu thập dataset cho bài toán Sarcasm detection in news headline**

Danh sách các trang tin đã chọn:

> Báo châm biếm:
> * The Hard Times: https://thehardtimes.net/
> * Weekly World News: https://weeklyworldnews.com/
> * The Daily Mash: https://www.thedailymash.co.uk/ 

> Báo chính thống:
> * Reuters: https://www.reuters.com/
> * Fox News: https://www.foxnews.com/world
> * wsj:https://www.wsj.com/

Thu thập tất cả tiêu đề của các bài báo mà 6 trang tin trên đăng trong vòng 03 năm trở lại đây

# **Add thư viện**

Giới thiệu thư viện BeautifulSoup

- BeautifulSoup là một thư viện Python dùng để lấy dữ liệu ra khỏi các file HTML và XML.

- BeautifulSoup hoạt động cùng với các trình phân tích cú pháp (parser), cung cấp cho người dùng các cách để điều hướng, tìm kiếm và chỉnh sửa trong các cây phân tích được tạo ra từ parser (parse tree).

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm

# **Thu thập dữ liệu**
# Báo châm biếm
- The Hard Times: https://thehardtimes.net/ --------------440 headlines

- Weekly World News: https://weeklyworldnews.com/ ---------4185 headlines

- The Daily Mash: https://www.thedailymash.co.uk/ ---------1000 headlines

> Tổng: 5625 headlines

Đối với các trang báo châm biếm, vì số lượng tin châm biếm không nhiều nên thường được tổng hợp luôn vào Page Home và phân số thứ tự trang. 

Sau khi tìm hiểu các trang tin, nhóm em nhận thấy mỗi lượt load tiếp trang mới, hoặc load thêm tin tức, thì web sẽ trả về dữ liệu thông qua một đường link có cấu trúc cố định, đồng thời mỗi headlines đều được lưu bên trong mã nguồn của web (F12), từ đó nhóm đã xem xét, crawl những headlines tin tức trong khoảng 03 năm của mỗi trang tin như sau


In [ ]:
# 1/ Weekly World News
weeklyworldnews = []
for i in range(44):
  url = 'https://weeklyworldnews.com/category/headlines/page/' + str(i+1)
  soup = BeautifulSoup(requests.get(url).content, "html.parser")
  titles = soup.findAll('h2', class_='entry-title')
  for link in titles:
    weeklyworldnews.append([link.find('a').attrs["href"], link.get_text(), 1])

In [ ]:
df_1 = pd.DataFrame(data=np.array(weeklyworldnews),columns=['article_link', 'headline','is_sarcastic'])

In [ ]:
# 2/ The hard times
thehardtimes = []
for i in tqdm(range(279), total=279):
  url = 'https://thehardtimes.net/page/' + str(i+1)
  soup = BeautifulSoup(requests.get(url).content, "html.parser")
  titles = soup.findAll('h2', class_='post-title')
  for link in titles:
    thehardtimes.append([link.find('a').attrs["href"], link.get_text(), 1])

100%|██████████| 279/279 [00:46<00:00,  5.99it/s]


In [ ]:
df_2 = pd.DataFrame(data=np.array(thehardtimes),columns=['article_link', 'headline','is_sarcastic'])

In [ ]:
# 3/ The Daily Mash, các mục về Chính trị, Đời sống, Thể thao
thedailymash = []
urls = ['https://www.thedailymash.co.uk/politics?page=','https://www.thedailymash.co.uk/news/lifestyle?page=','https://www.thedailymash.co.uk/sport?page=']
i_page = [20,77,3]
for ur, iter in tqdm(zip(urls, i_page) ,total=3):
  for i in range(iter):
    url = ur + str(i)
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    titles = soup.findAll('a',class_='font-serif font-bold text-xl xl:text-2xl text-brand hover:underline leading-none xl:leading-tight')
    for link in titles:
      thedailymash.append(['https://www.thedailymash.co.uk'+link.get('href'), link.get_text(), 1])


100%|██████████| 3/3 [01:20<00:00, 26.93s/it]


In [ ]:
df_3 = pd.DataFrame(data=np.array(thedailymash),columns=['article_link', 'headline','is_sarcastic'])

# **Thu thập dữ liệu**
# Báo chính thống
- Reuters: https://www.reuters.com/ --------- 28222 headlines

- Fox News: https://www.foxnews.com/world --------- 79920 headlines

- wsj: https://www.wsj.com/ ---------- 7600 headlines

> Tổng: 115742 headlines

Đối với các trang báo chính thống, vì số lượng tin chính thống rất nhiều và đa dạng, nên tin tức thường được chia thành các mục lớn. Vì vậy mà nhóm em đã chọn lọc, crawl headlines một số mục cụ thể của các trang tin

Sau khi xem xét, nhóm em nhận thấy một số web tin chính thống khi load thêm tin đã trả về dữ liệu thông qua hình thức call Api. Dựa vào thông tin dữ liệu thể hiện dưới dạng file Json, tìm kiếm headlines thông qua các đường dẫn của key từ đó crawl những headlines tin tức trong khoảng 03 năm của mỗi trang tin như sau

In [ ]:
# 1/ Reuters
# Crawl headlines các mục 'africa', 'americas', 'asia-pacific', 'china', 'europe', 'india', 'middle-east', 'uk', 'us', 'the-great-reboot'
place = ['africa', 'americas', 'asia-pacific', 'china', 'europe', 'india', 'middle-east', 'uk', 'us', 'the-great-reboot']

In [ ]:
# Request data
is_q = []
for p in place:
  for i in tqdm(range(1000), position=0, leave=True,total=1000):
    data = requests.get('https://www.reuters.com/pf/api/v3/content/fetch/articles-by-section-alias-or-id-v1?query=%7B%22id%22%3A%22%2Fworld%2F'+ p +'%22%2C%22offset%22%3A'+str(i*10)+'%2C%22orderby%22%3A%22last_updated_date%3Adesc%22%2C%22size%22%3A10%2C%22website%22%3A%22reuters%22%7D&d=35&_website=reuters')
    if data.status_code == 400:
      is_q.append((i-1)*10)
      break

  1%|▏         | 13/1000 [00:13<17:44,  1.08s/it]

In [ ]:
# Một data request từ trang Reuter mục africa, thể hiện dưới dạng file Json
data = requests.get('https://www.reuters.com/pf/api/v3/content/fetch/articles-by-section-alias-or-id-v1?query=%7B%22id%22%3A%22%2Fworld%2F'+ 'africa' +'%22%2C%22offset%22%3A'+str(2*10)+'%2C%22orderby%22%3A%22last_updated_date%3Adesc%22%2C%22size%22%3A10%2C%22website%22%3A%22reuters%22%7D&d=35&_website=reuters') 
r = data.json()
r

{'_id': '468b059e987194c6d411047566f2716b52ad82d5cefa2b782e5d3cbbe67424ee',
 'message': 'Success',
 'result': {'articles': [{'ads_primary_section': {'ad_unit_code': '/technology',
     'id': '/technology',
     'name': 'Technology',
     'path': '/technology',
     'website': 'reuters'},
    'authors': [{'name': 'Reuters', 'topic_url': '/authors/reuters/'}],
    'canonical_url': '/world/africa/nigeria-orders-broadcasters-not-use-twitter-gather-information-2021-06-07/',
    'content_code': 'metered',
    'description': "Nigerian television and radio stations should not use Twitter to gather information and have to de-activate their accounts, the broadcast authority said following the move to suspend the U.S. social media giant in Africa's most populous country.",
    'display_time': '2021-06-07T23:35:06.530Z',
    'id': 'VXMVHDZFVZM7VGEQ3VM7WVWNEY',
    'primary_media_type': 'gallery',
    'primary_section': {'ad_unit_code': '/africa',
     'id': '/world/africa',
     'name': 'Africa',


In [ ]:
# Số lượng headlines của từng mục
for p, i in zip(place, is_q):
  print('{} : {}'.format(p,i))

africa : 920
americas : 2920
asia-pacific : 6870
china : 5400
europe : 4000
india : 2470
middle-east : 3600
uk : 4300
us : 4610
the-great-reboot : 120


In [ ]:
# Get headlines
reuter = []
for p, iq in zip(place, is_q):
  print(p)
  for i in range(int((iq/10)+1)):
    try:
      data = requests.get('https://www.reuters.com/pf/api/v3/content/fetch/articles-by-section-alias-or-id-v1?query=%7B%22id%22%3A%22%2Fworld%2F'+ p +'%22%2C%22offset%22%3A'+str(i*10)+'%2C%22orderby%22%3A%22last_updated_date%3Adesc%22%2C%22size%22%3A10%2C%22website%22%3A%22reuters%22%7D&d=35&_website=reuters') 
      r = data.json()
    except:
      print('error: ',i,p)
    for u in range(10):
      try:
        reuter.append(['https://www.reuters.com'+r['result']['articles'][u]['canonical_url'], r['result']['articles'][u]['title'],0])
      except:
        print(p,i,u)

In [ ]:
df_4 = pd.DataFrame(data=np.array(reuter),columns=['article_link', 'headline','is_sarcastic'])

In [ ]:
# 2/ FoxNews
# Crawl headlines các mục 'us','world','politics','entertainment','lifestyle','science','tech','health'
pla = ['us','world','politics','entertainment','lifestyle','science','tech','health']
is_q = []
for p in pla:
  for i in range(1000):
    try:
      data = requests.get('https://www.foxnews.com/api/article-search?isCategory=true&searchSelected=fox-news%2F'+ p + '&tags=&size=10&offset=' + str(10*(i+1)))
      if not data.json():
        is_q.append((i)*10)
        print(p,i)
        break
    except:
      print('error :',p,i)

In [ ]:
# Một data request từ trang Foxnews mục world, thể hiện dưới dạng file Json
data = requests.get('https://www.foxnews.com/api/article-search?isCategory=true&searchSelected=fox-news%2F'+ 'world' + '&tags=&size=10&offset=' + str(10*(2+1))) 
r = data.json()
r

[{'category': {'name': 'China', 'url': '/category/world/world-regions/china'},
  'description': "No one attended an annual candlelight vigil inside Hong Kong’s Victoria Park on Friday, but hundreds gathered on streets outside to remember China's deadly crackdown in Beijing's Tiananmen Square in 1989 as Hong Kong police cordoned off much of the park to prevent gatherings.",
  'duration': '',
  'imageUrl': 'https://a57.foxnews.com/static.foxnews.com/foxnews.com/content/uploads/2021/06/348/196/candle-lit-2.jpeg?ve=1&tl=1',
  'isBreaking': False,
  'isLive': False,
  'lastPublishedDate': '2021-06-04T13:51:30-04:00',
  'publicationDate': '2021-06-04T13:51:30-04:00',
  'title': "China's deadly Tiananmen crackdown remembered by hundreds in Hong Kong despite ban",
  'url': '/world/tiananmen-hong-kong-vigil-ban'},
 {'category': {'name': 'Russia',
   'url': '/category/world/world-regions/russia'},
  'description': 'A Russian fighter jet deployed and accompanied a U.S. military plane over the Bar

In [ ]:
# # Get headlines
foxnews = []
for p, iq in zip(pla, is_q):
  print(p)
  for i in range(int((iq/10)+1)):
    try:
      data = requests.get('https://www.foxnews.com/api/article-search?isCategory=true&searchSelected=fox-news%2F'+ p + '&tags=&size=10&offset=' + str(10*(i+1))) 
      r = data.json()
    except:
      print('error: ',i,p)
    for u in range(10):
      try:
        foxnews.append(['https://www.foxnews.com'+ r[u]['url'], r[u]['title'],0])
      except:
        print(p,i,u)

In [ ]:
df_5 = pd.DataFrame(data=np.array(foxnews),columns=['article_link', 'headline','is_sarcastic'])

In [ ]:
# 3/ WSJ
wsj = []
plac = ['africa','asia','canada','china','europe','latin-america','middle-east']
it = [17, 75, 20, 51, 79, 56, 82]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for p, i in zip(plac, it):
  for u in range(i):
    while True:
      url = 'https://www.wsj.com/news/types/'+ p +'-news?page=' + str(u+1)
      soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")
      titles = soup.findAll('h2',class_='WSJTheme--headline--unZqjb45')
      if titles :
        break
    for link in titles:
      wsj.append([link.find('a').attrs['href'], link.get_text(), 0])

In [ ]:
df_6 = pd.DataFrame(data=np.array(wsj),columns=['article_link', 'headline','is_sarcastic'])

# **Tổng kết**
Tiến hành loại bỏ các headlines trùng lặp và xuất file csv
> Tổng 114717 headlines

In [ ]:
# Tổng hợp các headlines của 06 trang tin
bigdata = pd.concat([df_1,df_2, df_3, df_4, df_5, df_6,], ignore_index=True)

In [ ]:
bigdata = bigdata.drop_duplicates(subset=['headline'], keep='first')
bigdata = bigdata.drop(columns=['Unnamed: 0'])
bigdata

,article_link,headline,is_sarcastic
0,https://weeklyworldnews.com/headlines/182114/w...,WOMAN GIVES BIRTH TO CHERUB!,1
1,https://weeklyworldnews.com/headlines/182087/b...,BLONDIE SONG “RAPTURE” PREDICTED ALIEN ATTACK!,1
2,https://weeklyworldnews.com/headlines/182079/h...,HOW TO TELL IF YOU HAVE PSYCHIC POWERS,1
3,https://weeklyworldnews.com/headlines/182070/n...,NY METS ACE JACOB deGROM IS AN ALIEN!,1
4,https://weeklyworldnews.com/aliens/182055/alie...,ALIENS MAKING DEPOSITS IN OUR SPERM BANKS!,1
...,...,...,...
121362,https://www.wsj.com/articles/u-n-vote-rebukes-...,U.N. Vote Rebukes U.S. for Jerusalem Move,0
121363,https://www.wsj.com/articles/israel-rejects-u-...,Israel Rejects U.N. Vote as Palestinians Hail It,0
121364,https://www.wsj.com/articles/trump-threatens-c...,Trump Threatens Countries That Oppose His Deci...,0
121365,https://www.wsj.com/articles/saudi-arabia-expe...,Saudis Plan to Reap Billions in Crackdown on W...,0


In [ ]:
bigdata.to_csv('/content/drive/MyDrive/ML_Raw/news/ori.csv')